# Galvo Target Analysis
This notebook performs analysis of galvo targeting data using the GalvoTargetAnalysis class.

In [ ]:
import numpy as np
from process_spirals import GalvoTargetAnalysis
import os
import matplotlib.pyplot as plt
%matplotlib inline

## Initialize Analyzer and Get Directories

In [ ]:
# Initialize analyzer
analyzer = GalvoTargetAnalysis()

# Get directories through GUI
analyzer.get_directories()

## Set Parameters and Load Data

In [ ]:
# Set number of channels
analyzer.nchannels = 2  # Adjust if different

# Load Suite2p data
analyzer.load_suite2p_data()
print("Suite2p data loaded successfully")

## Get Frame Timing Information
This cell will show plots for verification. Press Enter in the output to continue through the plots.

In [ ]:
# Get frame timing information
analyzer.get_frame_times(channel_number=5, plot_on=True)

## Get Spiral Locations and Timing

In [ ]:
# Get spiral locations and timing
analyzer.get_spiral_locations(min_height=0.5, min_prom=0.1)

## Process Targets and Trials

In [ ]:
# Set trial window parameters
pre_trial_window = 30
post_trial_window = 30

# Process targets and trials
analyzer.process_targets_and_trials(
    pre_trial_window=pre_trial_window,
    post_trial_window=post_trial_window
)

## Create ROI Activity Plots

In [ ]:
# Create save directory for ROI plots
roi_plot_dir = os.path.join(analyzer.save_folder, 'roi plots')
os.makedirs(roi_plot_dir, exist_ok=True)

# Process each ROI
F_struct = {}
F_struct2 = {}
Fneu_struct = {}

for roi in range(analyzer.F.shape[0]):
    for target_id in range(len(analyzer.alignment_info[-1]['points'])):
        trial_frames = analyzer.trial_frames[target_id]
        
        if target_id not in F_struct:
            F_struct[target_id] = []
            F_struct2[target_id] = []
            Fneu_struct[target_id] = []
        
        for tr in range(trial_frames.shape[0]):
            if max(trial_frames[tr]) < analyzer.F.shape[1]:
                frames = trial_frames[tr].astype(int)
                
                # Store different versions of fluorescence traces
                F_trace = analyzer.F[roi, frames]
                Fneu_trace = analyzer.Fneu[roi, frames]
                F_corrected = F_trace - 0.7 * Fneu_trace
                
                F_struct[target_id].append(F_trace)
                F_struct2[target_id].append(F_corrected)
                Fneu_struct[target_id].append(Fneu_trace)

## Plot and Save ROI Activity

In [ ]:
# Create plots
for i in range(len(analyzer.alignment_info[-1]['points'])):
    plt.figure(figsize=(15, 5))
    
    # Sort by distance
    _, order = np.argsort(analyzer.dist_id[i])
    
    for n, idx in enumerate(order):
        plt.subplot(1, len(analyzer.target_id[i]), n+1)
        
        # Plot individual trials and mean
        traces = np.array(F_struct2[i])[:, :, analyzer.target_id[i][idx]]
        plt.plot(traces.T, 'gray', alpha=0.3)
        plt.plot(np.mean(traces, axis=0), 'k', linewidth=2)
        
        plt.title(f'ROI {analyzer.target_id[i][idx]}')
    
    plt.tight_layout()
    plt.savefig(os.path.join(roi_plot_dir, f'{i+1} candidate rois neu correct'))
    plt.show()

## Analysis Complete
Check the save folder for results and generated plots.